# Colored pointcloud to mesh using open3d
- http://www.open3d.org/docs/release/index.html 
- https://pymesh.readthedocs.io/en/latest/ ? 

In [6]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

pcd = o3d.io.read_point_cloud("test_data/fragment.ply")
print(pcd)
print(np.asarray(pcd.points))

def visualize(pcd):
    o3d.visualization.draw_geometries([pcd],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])
visualize(pcd)

PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


In [11]:
#voxelization downsample
DOWN_VOX_SIZE = 0.02
downpcd = pcd.voxel_down_sample(voxel_size=DOWN_VOX_SIZE)
print(downpcd)
visualize(downpcd)
pcd = downpcd

#TODO:
#downsampling reduces the quality of outcoming material
#how downsample mesh, but produce HighRes mat??

PointCloud with 27012 points.


# Meshing 
- http://www.open3d.org/docs/release/tutorial/geometry/surface_reconstruction.html


In [ ]:
#alpha shapes
alpha = 0.1
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [17]:
#ball pivoting
radii = [0.005, 0.01, 0.02, 0.04] #radii of individual balls 
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [12]:
#poisson s. reconstruction
depth = 11 #higher -> higher the detail
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

[Open3D DEBUG] Input Points / Samples: 27012 / 26803
[Open3D DEBUG] #   Got kernel density: 0.118447 (s), 534.539 (MB) / 534.539 (MB) / 534 (MB)
[Open3D DEBUG] #     Got normal field: 0.152304 (s), 534.539 (MB) / 534.539 (MB) / 534 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.857580e-05 / 1.042009e+00
[Open3D DEBUG] #       Finalized tree: 0.103259 (s), 534.539 (MB) / 534.539 (MB) / 534 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.156914 (s), 534.539 (MB) / 534.539 (MB) / 534 (MB)
[Open3D DEBUG] #Set point constraints: 0.0281229 (s), 534.539 (MB) / 534.539 (MB) / 534 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 542256 / 418816 / 200905
[Open3D DEBUG] Memory Usage: 534.539 MB
Cycle[0] Depth[0/9]:	Updated constraints / Got system / Solved in:  0.000 /  0.000 /  0.000	(320.449 MB)	Nodes: 8
CG: 5.6727e-01 -> 5.6727e-01 -> 1.4331e-07 (2.5e-07) [21966]
Cycle[0] Depth[1/9]:	Updated constraints / Got system / Solved in:  0.001 /  0.000 /  0.000	(320.449 MB)	Nodes: 27
 

In [13]:
#Ad poisson s. recon. --Remove low density vertices
#mesh density visualization more yellow more dense
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

visualize densities


In [14]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.1)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

remove low density vertices
TriangleMesh with 57974 points and 113826 triangles.


## Remove outliers?

In [10]:
visualize(mesh)
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)

## Denoising
- http://www.open3d.org/docs/release/tutorial/geometry/mesh.html#Average-filter 

In [14]:
#average filter
mesh_out = mesh.filter_smooth_simple(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

In [33]:
#laplacian filter
mesh_out = mesh.filter_smooth_laplacian(number_of_iterations=5)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

In [32]:
#taubin filter
mesh_out = mesh.filter_smooth_taubin(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out], mesh_show_wireframe=False)

## Mesh simplification
- http://www.open3d.org/docs/release/tutorial/geometry/mesh.html#Mesh-simplification

In [34]:
o3d.visualization.draw_geometries([mesh_out], mesh_show_wireframe=True)

In [ ]:
#vertex clustering - voxelization
mesh_in = mesh
voxel_size = max(mesh_in.get_max_bound() - mesh_in.get_min_bound()) / 128
print(f'voxel_size = {voxel_size:e}')
mesh_smp = mesh_in.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp], mesh_show_wireframe=True)

In [16]:
#mesh decimation
mesh_smp = mesh.simplify_quadric_decimation(target_number_of_triangles=6500)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)
o3d.visualization.draw_geometries([mesh_smp], mesh_show_wireframe=True)

Simplified mesh has 3530 vertices and 6499 triangles


In [ ]:
visualize(mesh_smp)

## Export

In [39]:
o3d.io.write_triangle_mesh("export/mesh_smp.obj",
                               mesh_smp,
                               write_triangle_uvs=True)

o3d.io.write_triangle_mesh("export/mesh.obj",
                               mesh,
                               write_triangle_uvs=True,
                               print_progress=True
                               )

> 1.7441e-02 (1.2e-02) [8]
Cycle[0] Depth[5/9]:	Updated constraints / Got system / Solved in:  0.002 /  0.014 /  0.011	(668.547 MB)	Nodes: 35937
          GS: 1.3063e+00 -> 1.3063e+00 -> 1.5695e-02 (1.2e-02) [8]
Cycle[0] Depth[6/9]:	Updated constraints / Got system / Solved in:  0.002 /  0.009 /  0.011	(668.547 MB)	Nodes: 37312
            GS: 1.1202e+00 -> 1.1202e+00 -> 7.5496e-03 (6.7e-03) [8]
Cycle[0] Depth[7/9]:	Updated constraints / Got system / Solved in:  0.004 /  0.021 /  0.027	(668.547 MB)	Nodes: 107760
              GS: 6.7318e-01 -> 6.7318e-01 -> 3.5445e-03 (5.3e-03) [8]
Cycle[0] Depth[8/9]:	Updated constraints / Got system / Solved in:  0.007 /  0.041 /  0.047	(668.547 MB)	Nodes: 206344
                GS: 2.7517e-01 -> 2.7517e-01 -> 3.7037e-04 (1.3e-03) [8]
Cycle[0] Depth[9/9]:	Updated constraints / Got system / Solved in:  0.003 /  0.004 /  0.001	(668.547 MB)	Nodes: 0
                  GS: 0.0000e+00 -> 0.0000e+00 -> 0.0000e+00 (-nan) [8]
Cycle[0] Depth[0/9]:	Updated cons

True